RDKitで計算可能な記述子を計算する。[計算可能な記述子一覧](http://www.rdkit.org/docs/GettingStartedInPython.html#list-of-available-descriptors)

In [1]:
import numpy as np
import pandas as pd
import sys

from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.rdBase import rdkitVersion


print(sys.version_info)
print(f'rdkit version: {rdkitVersion}')

sys.version_info(major=3, minor=6, micro=2, releaselevel='final', serial=0)
rdkit version: 2017.03.3


### 1. 記述子を計算させる分子の読み込み

#### 1.1 SDF

In [2]:
mols = [mol for mol in Chem.SDMolSupplier('input.sdf') if mol is not None]
mols[:5]

#### 1.2 SMILES

In [3]:
mols = [mol for mol in Chem.SmilesMolSupplier('input.smi', titleLine=False) if mol is not None]
mols[:5]

#### 1.3 直接SMILESを書く

In [4]:
smiles_set = [
    'c1ccccc1',
    'c1ccccc1Cl',
    'c1ccccc1Br',
    'C1CCCCC1',
    'C1CCCC1'
]
mols = [Chem.MolFromSmiles(smiles) for smiles in smiles_set]
mols[:5]

### 2. 計算する記述子の選択

計算可能な記述子の名前一覧は`Descriptors._descList`に関数と一緒に保存されている。

In [5]:
Descriptors._descList[:5]

[('MolWt', <function rdkit.Chem.Descriptors.<lambda>>),
 ('HeavyAtomMolWt', <function rdkit.Chem.Descriptors.<lambda>>),
 ('ExactMolWt', <function rdkit.Chem.Descriptors.<lambda>>),
 ('NumValenceElectrons',
  <function rdkit.Chem.Descriptors.NumValenceElectrons>),
 ('NumRadicalElectrons',
  <function rdkit.Chem.Descriptors.NumRadicalElectrons>)]

In [6]:
len(Descriptors._descList)

196

RDKit (version 2017.03.3) では、全部で196個の記述子を計算することができる。  
さらに以下のように実行することで名前だけ取り出すことができる。

In [7]:
names = [x[0] for x in Descriptors._descList]
names[:5]

['MolWt',
 'HeavyAtomMolWt',
 'ExactMolWt',
 'NumValenceElectrons',
 'NumRadicalElectrons']

`names`から計算する必要がない記述子があるならそれを取り除く。今回は先頭の5個だけ計算してみる。  

In [8]:
names = names[:5]
names

['MolWt',
 'HeavyAtomMolWt',
 'ExactMolWt',
 'NumValenceElectrons',
 'NumRadicalElectrons']

### 3. 記述子計算

#### 3.1 複数の記述子を一括で計算する

複数の記述子をまとめて計算するには`MoleculeDescriptors`モジュールの[MolecularDescriptorCalculator](http://www.rdkit.org/Python_Docs/rdkit.ML.Descriptors.MoleculeDescriptors.MolecularDescriptorCalculator-class.html)クラスを用いる。

In [9]:
calc = MoleculeDescriptors.MolecularDescriptorCalculator(names)
descs = [calc.CalcDescriptors(mol) for mol in mols]
descs

[(78.11399999999999, 72.06599999999999, 78.046950192, 30, 0),
 (112.559, 107.51899999999999, 112.00797784, 36, 0),
 (157.01, 151.96999999999997, 155.95746226, 36, 0),
 (84.162, 72.06599999999999, 84.093900384, 36, 0),
 (70.135, 60.05499999999999, 70.07825032, 30, 0)]

機械学習などで用いるためにデータフレームに変換してみる。

In [10]:
df = pd.DataFrame(descs, columns=names)
df.head()

,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons
0,78.114,72.066,78.046950,30,0
1,112.559,107.519,112.007978,36,0
2,157.010,151.970,155.957462,36,0
3,84.162,72.066,84.093900,36,0
4,70.135,60.055,70.078250,30,0


さらにcsvファイルなどで保存したい場合は`to_csv`メソッドを用いればよい。  
今回の例にはないが化合物の名前にはカンマを含むことがしばしばあるため、カンマ区切りではなくタブ区切りなどをしたほうがよい。  
以下にタブ区切りで出力する例を示す。

In [11]:
df.to_csv('desc.csv', sep='\t')

カレントディレクトリの`desc.csv`に計算した記述子が出力される。

#### 3.2 一つずつ記述子を計算する

一つずつ記述子を計算するには`Descriptors`モジュールの各関数を用いればよい。  
例えば、分子量を計算する場合は以下のようになる。

In [12]:
mol = Chem.MolFromSmiles('c1ccccc1')
Descriptors.MolWt(mol)

78.11399999999999